[Selection de carac.](https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e) ; [SKLearn - Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html)

[SKLearn - KBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) ; [SKLearn - Percentile](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile)


In [4]:
#Code repris pour d'autres choses, là on observe seulement le paramètre max_df du TF-IDF à 50%
#Ce qui signifie qu'on "écréme" 50% des données

import pickle
import cPickle
import numpy

#from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import train_test_split


def preprocess(words_file = "../tools/word_data.pkl", authors_file="../tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    #features_train, features_test, labels_train,\ 
    #    labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)
    features_train, features_test, labels_train,\
        labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=1)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

preprocess()
print ''

no. of Chris training emails: 7936
no. of Sara training emails: 7884



Rappels :

[TF-IDF](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting) ; [TF-IDF - How-To](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

[Dilemme biais-variance](https://openclassrooms.com/fr/courses/4011851-initiez-vous-au-machine-learning/4092326-trouvez-le-bon-compromis-entre-biais-et-variance) ; On revient sur un concept déjà vu et qu'on peut réduire grossièrement (dans le contexte de selection de feature) en : biai = Pas assez de features / variance = Trop de feature

[Cours complet sur la Régression linéaire (+ la régularisation)](https://openclassrooms.com/fr/courses/4444646-entrainez-un-modele-predictif-lineaire)

Attention on ne parle ici que en contexte apprentissage supervisé. Pour rappel :
Supervisé -> clf.fit(features, labels) /
Non-supervisé -> clf.fit(features)

[SKLearn & la régularisation]

https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression-and-classification

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

In [28]:
import pickle
import numpy
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pprint
numpy.random.seed(42)


### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "./your_word_data.pkl" 
authors_file = "./your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )



### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier

features_train, features_test, labels_train,\
    labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

feat_names = vectorizer.get_feature_names()


### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]



### your code goes here
clf_dt = tree.DecisionTreeClassifier()
clf_dt = clf_dt.fit(features_train, labels_train)
pred = clf_dt.predict(features_test)
acc = accuracy_score(labels_test, pred)

feat_imp = clf_dt.feature_importances_
cleaned_feat_imp = dict()

i = 0
for coef in feat_imp:
    if coef != 0.:
        cleaned_feat_imp[i] = coef
    i+=1


print acc
print len(features_train)
pprint.pprint(cleaned_feat_imp)
print(feat_names[21150])

#En fait, on va repérer les signatures des mails qui évidemment, orientent beaucoup trop efficacement
#sur l'identité de l'auteur...

0.842434584755
150
{19203: 0.047876569569416959,
 19399: 0.024953292321713439,
 19852: 0.025618713450292402,
 21150: 0.27118644067796616,
 21163: 0.29131355932203384,
 22366: 0.10416666666666667,
 29414: 0.077519379844961184,
 30028: 0.026311111111111063,
 33571: 0.02,
 33659: 0.063042869151022723,
 33957: 0.048011397884815565}
houect
